# Processing Raw Data with _ExpressYeaself_

### Introduction

* This **interactive** notebook that will **automate** the **processing** of raw data. All you need to do is **set the parameters** that control the way in which the data is processed.  


* If you haven't already done so, please **download the raw data** by following the installation instructions found in our [README](https://github.com/yeastpro/ExpressYeaself/blob/master/README.md).  


* Run (using ``shift`` + ``enter``) **every cell** in this notebook from top to bottom . You'll need to **input some arguments** for some functions where instructed to before running the cell. This will involve assigning values to variables but typing some input after ``=`` signs.  


* If an error is thrown, check your input and try to run the cell again. Make sure you've assigned the variables by typing exactly what's in the ``codeblock``. For example, for parameter 2 ``'pTpA'`` is a correct assignment but ``pTpA`` **is not**.

### Importing some packages

In [13]:
import context
import os

process = context.process_data

### Defining the paths to the raw data

In [14]:
current_directory = os.getcwd()
raw_pTpA = current_directory + '/../../example/pTpA_data/raw_data_pTpA.txt.gz'
raw_data_Abf1TATA = current_directory + '/../../example/Abf1TATA_data/raw_data_Abf1TATA.txt.gz' 

### Choosing the processing parameters

1. Decide what raw data you want to use. Type ``raw_pTpA`` or ``raw_Abf1TATA`` after the ``=`` sign.

In [15]:
raw_data = raw_pTpA

2. Choose the scaffold type. If you chose ``raw_pTpA`` the scaffold type is ``'pTpA'`` and if you chose ``raw_Abf1TATA`` the scaffold type is ``'Abf1TATA'``.

In [1]:
scaffold_type = 'pTpA'

3. If you specify a value for this parameter, the sequences in your raw data file are sorted by expression level. The top and bottom percentiles of the data are then extracted and proceed with the data processing, whereas the middle portion of the data is discarded.
    * For example, if you specify ``percentile = 0.25`` the quarter of sequences with the highest expression levels and the quarter of sequences with the lowest expression levels are extracted. The middle 50 % of data is discarded.
    * **Why use this?** This parameter is useful for creating extremes of data based on expression level, which can be used to train a classification model. This can predict the probability that a sequence will express _high_ or express _low_.
    * **Why not?** Is best used to train binary classification models. For a more quantitative prediction of expression level across a whole range, set this parameter to ``None``.

In [2]:
percentile = 0.25

4. If (and only if) you have set a value of ``percentile`` that **is not ``None``**, choose whether or not to binarize the expression levels. This will set the expression levels of all sequences in the top pecentile to ``1`` and all the expression levels in the bottom percentile to ``0``. 
    * Highly recommended that you set ``binarize_expression_levels = True`` if you have specified a value for ``percentile``.
    * Otherwise, set ``binarize_expression_levels = False``.

In [3]:
binarize_expression_levels = True

5. The raw data you have downloaded contains sequences of **variable** length, ranging from 97 to 127 nucleotides. To train a neural network model, the inputs must be encoded and all the encoded sequences must be the same length. Sequences are **automatically padded** so they are the same length. **However**, if you choose to set ``pull_only_homogeneous = True`` all the sequences that have the modal (most common) length will be pulled out. Every sequence will by definition have the same length - be _homogeneous_ - so will not need padding. For pTpA data, for instance, this is 110 nucleotides, and for Abf1TATA data this is 115 nucleotides.
    * If you choose ``pull_only_homogeneous = False``, sequences that are shorter than the longest sequence in the file will be 'padded' to the max length. When you encode your data, the padding will be encoded as empty vectors.  

In [4]:
pull_only_homogeneous = True

6. The sequences in the raw data file contain '**flanking regions**'. This are short nucleotide sequences on each end of the oligonucleotide sequences found in the file that aid in the synthesis of polynucleotide sequences where nucleotides are inserted into '**scaffold sequences**'. Every sequence in each raw data file has the same flanking regions, though the flanking regions in the pTpA sequence data are **different** than those in the Abf1TATA sequence data. Here you can choose whether or not to remove these flanking regions.
    * Recommended: ``deflank_sequences = True`` as we can remove as many constants as possible before training a model that needs to pick up on subtleties.

In [5]:
deflank_sequences = True

7. Here you can choose whether or not to insert the sequences found in the raw data file into the middle of their corresponding scaffold sequences. Set ``insert_into_scaffold = True`` or ``insert_into_scaffold = False``.

In [6]:
insert_into_scaffold = True

8. Here you can choose whether or not to add extra padding to the sequences. This may be useful if you want to increase the sequence length to a particular length, but otherwise is not recommended; the automatic padding mechanism (or selecting ``pull_only_homogeneous = True`` in step 5) is usually sufficient. Set ``extra_padding = 0`` if you don't want to add extra padding, or else put in another positive integer if you do. Setting ``extra_padding = 3`` will pad sequences by an extra 3 empty nucleotides (that will be encoded as empty vectors later).

In [7]:
extra_padding = 0

9. If you have selected to use some extra padding, or have set ``pull_only_homogeneous = False`` (which automatically pads sequences), here you can choose whether that padding is added to the front (LHS) or back (RHS) of the sequences. Set ``pad_front = True`` if you want to back the front, or ``pad_front = False`` if you want to pad the back. If you have set ``pull_only_homogeneous = True`` and ``extra_padding = 0`` it doens't matter what you set this paramter to as no padding will be applied (you must still set it to something though).

In [8]:
pad_front = False

10. If you would like a log report of the data processing to be written to file set ``log_process_report = True``. This will write **timings** of each step and **data lost/discarded** at each stage to file, which can be found at file path ``ExpressYeaself/example/processed_data/``. It is recommended to set ``log_process_report = False`` as this information is also printed at the end of the notebook when you call the function.

In [9]:
log_process_report = False

11. If you would like the intermediate files created at each step of the data processing to be deleted after the process is complete, set ``remove_files = True``. It is **strongly recommended** to do so. This is because we are dealing with very large files, so having files for each step will use up a lot of memory. Otherwise, set ``remove_files = False``.

In [10]:
remove_files = True

12. Finally, if you would like to create a smaller sample data file based on your processed data, then you can specify a sample size here. This will pull this many sequence and expression level data lines from your processed data file pseudo-randomly.
    * This is useful for playing about with model architectures as a smaller data set (recommended size: ``sample_size = 10_000``) will run significantly faster.
    * If you don't want to create a sample data file, set ``sample_size = None``

In [11]:
sample_size = 10_000

### Calling the function

Now you have specified all the parameters you are ready to call the function that processes the raw data. Just run the following cell and wait ~ 10 minutes (depending on computer performance).

In [16]:
processed_data = process.process_raw_data(input_seqs=raw_data,
                                          scaffold_type=scaffold_type,
                                          percentile=percentile,
                                          binarize_els=binarize_expression_levels,
                                          homogeneous=pull_only_homogeneous,
                                          deflank=deflank_sequences,
                                          insert_into_scaffold=insert_into_scaffold,
                                          extra_padding=extra_padding,
                                          pad_front=pad_front,
                                          report_loss=log_process_report,
                                          report_times=log_process_report,
                                          remove_files=remove_files,
                                          create_sample_of_size=sample_size)

Starting processing of raw data...
Pulling out the top and bottom percentiles...
Binarizing expression levels...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/joe.abbott/Documents/dataScience/capstone/ExpressYeaself/expressyeaself/interaction/example/processed_data/20190613184401614474_df_to_file.txt'

Great! Now you have the absolute path of the processed data you've just created; copy this, you'll need it in the next step when you build and train a model.

### End